# 第10回: RNN: Exercise2-A
## 手書き文字のストローク予測
### 概要
本演習ではchainerを用いて手書き文字のストローク予測を行います。

In [23]:
import time
import os
import pickle

import numpy as np
import chainer
from chainer import Variable, optimizers, serializers, cuda

from utils import DataLoader
from model_chainer import Model

In [11]:
data_loader = DataLoader(batch_size=50, seq_length=300, scale_factor=20.)

train data: 11035, valid data: 580


モデルの保存場所を`model_dir`に設定します。

ハイパーパラメータなどの設定`config.pkl`及び、モデルデータがこのディレクトリに保存されることになります。

In [12]:
model_dir = './models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
with open(os.path.join(model_dir, 'config.pkl'), 'wb') as f:
    pickle.dump('model_name', f)

モデルのインスタンスを作成します。

In [13]:
model = Model(nb_layers=2, nb_units=256,
              nb_mixtures=20, data_scale=20,
             dropout=0.8)

<div class="alert alert-block alert-info">
Note:<br>
以下でCPU or GPUを使うか選択してください。
</div>

In [14]:
# model.to_gpu(device=0)

Gradient clipping ADAM等の説明

In [17]:
chainer.config.use_cudnn = "never"
optimizer = optimizers.Adam(alpha=0.0)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.GradientClipping(threshold=10.))

In [21]:
train_epochs = 30
lr = 0.005  # learning rate
decay_rate = 0.95  # for rmsprop
gpu_id = 0
save_every = 500

In [22]:
for e in range(train_epochs):
    optimizer.aplha = lr * (decay_rate ** e)  ## RMS prop
    data_loader.reset_batch_pointer()  # too
    v_x, v_y = data_loader.validation_data()
    v_x = np.array(v_x)
    v_y = np.array(v_y)

    if gpu_id >= 0:
        v_x = cuda.to_gpu(v_x, gpu_id)
        v_y = cuda.to_gpu(v_y, gpu_id)
    
    sum_loss = 0
    for b in range(data_loader.num_batches):
        i = e * data_loader.num_batches + b
        start = time.time()  # get start time
        
        x, y = data_loader.next_batch()
        x = np.array(x)
        y = np.array(y)
        
        if gpu_id >= 0:
            x = cuda.to_gpu(x, gpu_id)
            y = cuda.to_gpu(y, gpu_id)

        
        chainer.config.train = True  # todo: check
        model.cleargrads()
        train_loss = model(Variable(x), Variable(y))
        train_loss.backward()
        train_loss.unchain_backward()
        optimizer.update()
        sum_loss += float(cuda.to_cpu(train_loss.data))
        
        # Forward Mode
        chainer.config.train = False
        valid_loss = float(
            cuda.to_cpu(model(Variable(v_x), Variable(v_y)).data)
        )
        
        end = time.time()  # get end time
        
        print("{}/{} (epoch {}), train_loss = {:.3f}, valid_loss = {:.3f}, time/batch = {:.3f}"
              .format(i, train_epochs * data_loader.num_batches,
                          e, sum_loss / (b + 1), valid_loss, end - start))
        
        if (e * data_loader.num_batches + b) % save_every == 0 and ((e * data_loader.num_batches + b) > 0):
                checkpoint_path = os.path.join(model_dir,
                                               'model_{}.npz'.format(i))
                serializers.save_npz(checkpoint_path, model)
                print("model saved to {}".format(checkpoint_path))

0/10920 (epoch 0), train_loss = 4.147, valid_loss = 4.183, time/batch = 0.950
1/10920 (epoch 0), train_loss = 4.092, valid_loss = 4.183, time/batch = 0.912
2/10920 (epoch 0), train_loss = 3.988, valid_loss = 4.183, time/batch = 0.906
3/10920 (epoch 0), train_loss = 4.059, valid_loss = 4.183, time/batch = 0.906
4/10920 (epoch 0), train_loss = 4.014, valid_loss = 4.183, time/batch = 0.947
5/10920 (epoch 0), train_loss = 4.015, valid_loss = 4.183, time/batch = 0.922
6/10920 (epoch 0), train_loss = 3.983, valid_loss = 4.183, time/batch = 0.914
7/10920 (epoch 0), train_loss = 4.004, valid_loss = 4.183, time/batch = 0.908
8/10920 (epoch 0), train_loss = 3.987, valid_loss = 4.183, time/batch = 0.908
9/10920 (epoch 0), train_loss = 4.043, valid_loss = 4.183, time/batch = 0.911
10/10920 (epoch 0), train_loss = 4.072, valid_loss = 4.183, time/batch = 0.916
11/10920 (epoch 0), train_loss = 4.050, valid_loss = 4.183, time/batch = 0.913
12/10920 (epoch 0), train_loss = 4.030, valid_loss = 4.183, ti

KeyboardInterrupt: 